# Práctica 2: Planificación de Trayectorias
En la práctica 1 realizamos la estimación de las variables de articulación para un robot de tipo RR utilizando el problema cinemático directo y el problema cinemático inverso. Gracias a ello, pudimos simular el movimiento de un robot a lo largo de una trayectoria rectilínea con velocidad constante, y observar las diferentes indeterminaciones que existían. 

En el mundo real, las trayectorias son más complejas que una línea a velocidad constante. Normalmente se requiere tener en cuenta que comenzamos a velocidad 0 y debemos acabar a la misma velocidad 0, y que la trayectoria no va a ser totalmente recta, puesto que muchas veces tenemos que sortear obstáculos. De este modo, se puede definir la **trayectoria 3-4-3** que vamos a definir en esta práctica. 

En esta práctica realizaremos un planificador de trayectorias de tipo 3-4-3, en las que las velocidades y aceleraciones inicial y final son nulas. Hemos definido este tipo de trayectorias en teoría, pero recordaremos que constan de tres fases, que interpolan los puntos de inicio, despegue, asentamiento y fin, que nombramos como $\mathbf{p}_{ini}$, $\mathbf{p}_{des}$, $\mathbf{p}_{ase}$ y $\mathbf{p}_{fin}$. 

Para ello: 
1. Usaremos las funciones `pcd` y `pci` de la práctica anterior, para poder calcular los parámetros del robot.
2. Definiremos la función `planif`, que tomará como entrada estos puntos, el tiempo de duración de cada uno de los trayectos y el periodo de muestreo (para la representación de la trayectoria). Esta función nos tiene que devolver un array con los coeficientes de cada una de las trayectorias. 
3. Evaluaremos $\theta_1(t)$ y $\theta_2(t)$ en cada instante usando dichos coeficientes, y mostraremos el robot utilizando la función `robot`y las animaciones que usamos en la práctica anterior. 
4. Mostraremos también gráficas con el progreso de las variables $\theta$, la velocidad y la aceleración del robot en cada instante de tiempo. 

In [ ]:
import numpy as np

En primer lugar, definiremos las posiciones de cada punto de inicio, despegue, asentamiento y fin ($\mathbf{p}_{ini}$, $\mathbf{p}_{des}$, $\mathbf{p}_{ase}$ y $\mathbf{p}_{fin}$).

In [ ]:
# Datos de inicio
pini = np.array([1, 0])
pdes = np.array([1, 0.1])
pase = np.array([1.5, 0.1])
pfin = np.array([1.5, 0])
t1, t2, tn = 1,1,1
L1, L2 = 1,1
T = 0.01

### Trayectoria 3-4-3
Como hemos visto en el tema 4, las trayectorias del tipo 4-3-4 siguen unas ecuaciones predefinidas. Para empezar, asumimos que: 
- En el punto de inicio, la aceleración y velocidad son nulas.
- En el punto final la velocidad y aceleración son nulas. 
- En los puntos intermedios debe haber continuidad en posición, velocidad y aceleración. 

A lo largo de la práctica podremos utilizar el tiempo normalizado: 

$$ t = \frac{\tau-\tau_{i-1}}{\tau_{i}-\tau_{i-1}} = \frac{\tau-\tau_{i-1}}{t_i} $$

Donde $\tau$ es el tiempo real en segundos, $\tau_i$ es el tiempo real al final del $i$-esimo intervalo y $t_i$ es el tiempo total requerido en dicho intervalo. 

Si fuera una sola línea, obtendríamos un polinomio de grado 8, dificil de resolver y que puede conducir a movimientos extraños y bruscos. Así pues, dividimos la trayectoria en tres segmentos con polinomios de grado 4, 3 y 4. Los polinomios que definen cada trayectoria ($h_1$, $h_2$ y $h_n$) son:

$$ h_1(t) = a_{14}t^4 + a_{13}t^3 + a_{12}t^2 + a_{11}t + a_{10} $$
$$ h_2(t) = a_{23}t^3 + a_{22}t^2 + a_{21}t + a_{20} $$
$$ h_n(t) = a_{n4}t^4 + a_{n3}t^3 + a_{n2}t^2 + a_{n1}t + a_{n0} $$

Hay algunos parámetros $a_{ij}$ que se derivan de una serie de ecuaciones de continuidad, y podéis encontrarlos en las transparencias. Para el resto necesitamos la ecuación matricial que define las condiciones (1) y (7): 

$$ \mathbf{y} = \mathbf{C}\mathbf{x}$$

con los vectores:

$$ \mathbf{y} = (\delta_1 - \frac{a_0t_1^2}{2}-v_0t_1, -a_0t1-v_0, -a_0, \delta_2, -a_ft_n + v_f, a_f, \delta_n + \frac{a_ft_n^2}{2}-v_ft_n)$$

(Nota: en la definición de y, los $a$ y $v$ son la aceleración y la velocidad en los puntos iniciales y final, no los parámetros $a$). 

$$\mathbf{x} = (a_{13}, a_{14}, a_{21}, a_{22}, a_{23}, a_{n3}, a_{n4})$$

Por lo que podemos obtener los parámetros como los diferentes componentes de $\mathbf{x}$:

$$ \mathbf{x} = \mathbf{C}^{-1}\mathbf{y}$$

Podéis encontrar la matriz $\mathbf{C}$ en las transparencias. ¡Cuidado con la posición de cada elemento!

### Resumiendo: 
1. Para obtener los valores de $\theta_1$ y $\theta_2$ tenemos que evaluar los polinomios $h_1$, $h_2$ y $h_n$ (definimos los tres polinomios para **cada** articulación) en el tiempo. 
2. Para calcular $h_1$, $h_2$ y $h_n$, debemos obtener todos los coeficientes $a_{ij}$. 
3. Para calcular los coeficientes $a_{ij}$ necesistamos conocer los valores de $\theta_i$, $a$ (aceleración) y $v$ (velocidad) al principio y al final de la trayectoria. Algunos irán en las fórmulas directas y otros en $\mathbf{C}$ e $\mathbf{y}$. 
4. Para calcular los $\theta_i$ en los puntos de inicio, despegue, asentamiento y fin haremos uso del `pci` de la práctica anterior. 

In [ ]:
# Copiar y pegar PCI anterior... 


In [ ]:
# Obtenemos los ángulos theta 1 y theta 2 en cada punto. Si es necesario elegir, 
# usaremos el par de soluciones positivo, o cuando thant == 0


Vamos al meollo de la cuestión. Hay que definir la función `planif`. Las entradas serán los diferentes $\theta$, y la salida deben ser los coeficientes de las tres ecuaciones de la trayectoria (habrá que ejecutar planif 2 veces, una para cada articulación). Ver Tema 4.2 de teoría, a partir de página 20. 

Se recomienda devolver los coeficientes como un array 2D, donde cada fila es una trayectoria (3 filas), y los coeficientes están ordenados de menor exponente a mayor (5 columnas). 

In [ ]:
# Aquí vamos a definir nuestra función planif. 
def planif(thini, thdes, thase, thfin, t1=1., t2=1., tn=1.):
    # Cuerpo de la función

In [ ]:
# Ahora, guardamos las ecuaciones de la trayectoria para cada articulación. 
coef1 = planif(thini[0],thdes[0],thase[0],thfin[0],t1,t2,tn)
coef2 = planif(thini[1],thdes[1],thase[1],thfin[1],t1,t2,tn)

print(coef1)
print(coef2)

Ahora evaluamos los polinomios en cada intervalo de tiempo usando `polyval`, y los concatenamos. Recordad que, por la formalización que hemos utilizado en `planif` (la que hemos estudiado en teoría), utilizamos el tiempo normalizado: 

$$ t = \frac{\tau-\tau_{i-1}}{\tau_i -\tau_{i-1}}$$

con $\tau$ el tiempo en segundos, $\tau_i$ el tiempo al final de la $i$-ésima trayectoria y $\tau_i-\tau_{i-1}$ el tiempo que toma en la iésima trayectoria. Asimismo, recuerda que en el último segmento realizamos un último cambio de variable $\hat{t}=t-1$, con lo que el rango de tiempo normalizado pasa de (0,1) a (-1,0). 

In [ ]:
from numpy.polynomial.polynomial import polyval

# Primero, evalua cada ángulo en el intervalo correspondiente

# Después, concatena todos los intervalos para tener un solo vector con los valores de cada angulo en función del tiempo

## Trazado de la Trayectoria
Ahora nos ponemos visuales. Vamos a representar el movimiento del robot y los diferentes puntos de la trayectoria. Usaremos `matplotlib.pyplot` para visualización y las herramientas que ya utilizamos anteriormente para dibujar la animación. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation
from IPython.display import HTML 

Aquí reutilizamos el código para pintar el robot utilizando PCD (probablemente se llamara `pintarPCD2`) y sus dependencias (`init`, el `robot` y `pcd` que corresponda). 

Recordad que ahora ya disponemos de los valores de theta en cada punto, de modo que habrá que modificar nuestra antigua función `pintarPCD` para que trabaje directamente con estos valores. 

In [ ]:
# Aquí es donde copiamos el código de la práctica anterior y modificamos pintarPCD.


In [ ]:
#Llamamos a los constructores de la animación (recordad que hemos definido T en la primera caja)
N = j1.shape[0]#longitud del vector donde hayamos guardado todos los theta_1 o theta_2
anim = animation.FuncAnimation(fig, pintarPCD2, frames = N, fargs = (L1, L2, j1, j2, ax), interval = .1/T) 
HTML(anim.to_html5_video())
#anim.save('video.mp4') # Si hay problemas con visualización inline.

## Visualización de las variables
Ahora mostraremos, en función del tiempo: 
1. Los valores de $\theta_1$ y $\theta_2$
2. Los valores de velocidad $v_1$ y $v_2$
3. Los valores de aceleración $a_1$ y $a_2$

Para obtener la velocidad y aceleración, sabemos que son la primera y segunda derivada de las funciones de trayectoria $h_1$, $h_2$ y $h_n$, así que utilizaremos `polyder` para derivar el polinomio correspondiente, junto con `polyval`, para evaluar estas derivadas. 

In [ ]:
# Dibuja aquí la progresión de los dos ángulos theta1 y theta2
t = np.arange(0, t1+t2+tn, T)


In [ ]:
# Aquí cargamos polyder y mostramos la velocidad
from numpy.polynomial.polynomial import polyder


In [ ]:
# Y aquí mostramos la aceleración (segunda derivada)